Firts, we need to import the relevant libraries:
- **pandas** for data analytics
- **time** for time manipulation functions
- **lxml** for working with HTML/XML text
- **requests** for scaping web pages

In [1]:
import pandas as pd 
import time as t
from lxml import html 
import requests

Assign an empty data frame to a variable called *reviews_df*

In [2]:
reviews_df = pd.DataFrame()

- Go to www.yelp.com. 
- Type in *Clemson, SC* in the **Near** box and click Search.
- Copy the content inside the URL and paste it into an empty cell. The pasted text will look like this:

https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1

- Each browser has a unique signature, presented by an attribute called *user agent*. Many websites do not like being scraped, and they distinguish between a browser visit and automated programs by *user agent*. 

- Open a browser and go to http://www.whatsmyua.info/?source=post_page---------------------------, then copy and paste content from the detected user-agent string box.

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15


With this information, we can setup the variables for a web scape of restautants in Philly

In [3]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'

In [4]:
headers = {'User-Agent': user_agent}

In [5]:
searchlink = 'https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1'

In [6]:
page = requests.get(searchlink, headers = headers)
parser = html.fromstring(page.content)

Find the links for the restaurants listed from the page

In [7]:
businesslink=parser.xpath('//a[@class="lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5"]')
links = [l.get('href') for l in businesslink]

In [8]:
for link in links:
    print (link)

/adredir?ad_business_id=TqjpftzjY8S5cSufT9oOIg&campaign_id=g8PX_TFRdvoRpyTUiwSHNg&click_origin=search_results&placement=above_search&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fclemson-wine-bar-clemson&request_id=8251eeb50a820bed&signature=1f5f654b19fcf71a43de679e7e6b8a8dc35ff54a448bd6fa5d40bdce5156bd4b&slot=0
/adredir?ad_business_id=TqjpftzjY8S5cSufT9oOIg&campaign_id=g8PX_TFRdvoRpyTUiwSHNg&click_origin=read_more&placement=above_search&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fclemson-wine-bar-clemson%3Fhrid%3DGNxhRWE0O3b8gLaClu64JA&request_id=8251eeb50a820bed&signature=241a663f2704556a325b0d00fcba1179ff1b78d5f3f519be755b39791eeb4a87&slot=0
/adredir?ad_business_id=P73t5YvJq252VY1-uDYuJw&campaign_id=AK5vDm-BGO7MYMkj0p6kpw&click_origin=search_results&placement=above_search&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Flos-poblanos-six-mile-2&request_id=8251eeb50a820bed&signature=8ed839959e3630a257464047cb3ceeef2ecd41b67949052063ef48f26459ab32&slot=1
/adredir?ad_business_id=P7

List comprehension (Pythonic way of thinking!)

In [9]:
true_links = [link for link in links if link.startswith('/biz') and \
              not "hrid" in link]
for link in true_links:
    print (link)

/biz/yolk-asian-kitchen-clemson-3
/biz/all-in-coffee-shop-clemson
/biz/pot-belly-deli-clemson
/biz/blue-heron-restaurant-and-sushi-bar-clemson
/biz/your-pie-clemson
/biz/spill-the-beans-clemson-clemson
/biz/bake-and-cook-thai-restaurant-clemson
/biz/calhoun-corners-restaurant-clemson
/biz/todaro-pizza-clemson
/biz/loose-change-clemson


We need to create the URLs for these restaurants

In [10]:
urls = []

for link in true_links:
    url = 'https://www.yelp.com' + str(link)
    print (url)
    urls.append(url)

https://www.yelp.com/biz/yolk-asian-kitchen-clemson-3
https://www.yelp.com/biz/all-in-coffee-shop-clemson
https://www.yelp.com/biz/pot-belly-deli-clemson
https://www.yelp.com/biz/blue-heron-restaurant-and-sushi-bar-clemson
https://www.yelp.com/biz/your-pie-clemson
https://www.yelp.com/biz/spill-the-beans-clemson-clemson
https://www.yelp.com/biz/bake-and-cook-thai-restaurant-clemson
https://www.yelp.com/biz/calhoun-corners-restaurant-clemson
https://www.yelp.com/biz/todaro-pizza-clemson
https://www.yelp.com/biz/loose-change-clemson


Copy/paste one of the links to a browser tab to see where it takes you. 

This is only 10. What about the rest of them? 

In [11]:
page_nums = '//span[@class="lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_"]'
pg = parser.xpath(page_nums)
for p in pg:
    print(p.text)

Page 1 of 9
None


In [12]:
print(pg[0].text)

Page 1 of 9


In [13]:
s = "www.wcupa.edu"
s.split('.')

['www', 'wcupa', 'edu']

In [14]:
t = 'golden ram'
t.split()

['golden', 'ram']

In [15]:
pg[0].text.split()

['Page', '1', 'of', '9']

In [16]:
int(pg[0].text.split()[3])

9

What is the pattern?

https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C%20SC&ns=1&start=10
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C%20SC&ns=1&start=20

Can we use + instead of %20?

Let's make up an URL and test our patterns:
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1&start=60


Putting everything together!

In [17]:
import pandas as pd 
import time as t
from lxml import html 
import requests

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'
headers = {'User-Agent': user_agent}
urls = []


page_count = int(pg[0].text.split()[3])
suffix = ""
for i in range(1,(page_count + 1)):
    searchlink = 'https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1' + suffix
    print(searchlink)
    page = requests.get(searchlink, headers = headers)
    parser = html.fromstring(page.content)
    businesslink=parser.xpath('//a[@class="lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5"]')
    links = [l.get('href') for l in businesslink]
    true_links = [link for link in links if link.startswith('/biz') and not "hrid" in link]
    for link in true_links:
        url = 'https://www.yelp.com' + str(link)
        urls.append(url)
    t.sleep(4)
    suffix = '&start=' + str(10 * i)

https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1&start=10
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1&start=20
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1&start=30
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1&start=40
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1&start=50
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1&start=60
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1&start=70
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1&start=80


In [18]:
print(len(urls))

82


In [19]:
for item in urls:
    print(item)

https://www.yelp.com/biz/yolk-asian-kitchen-clemson-3
https://www.yelp.com/biz/all-in-coffee-shop-clemson
https://www.yelp.com/biz/pot-belly-deli-clemson
https://www.yelp.com/biz/blue-heron-restaurant-and-sushi-bar-clemson
https://www.yelp.com/biz/your-pie-clemson
https://www.yelp.com/biz/spill-the-beans-clemson-clemson
https://www.yelp.com/biz/bake-and-cook-thai-restaurant-clemson
https://www.yelp.com/biz/calhoun-corners-restaurant-clemson
https://www.yelp.com/biz/todaro-pizza-clemson
https://www.yelp.com/biz/loose-change-clemson
https://www.yelp.com/biz/zen-ramen-and-sushi-burrito-clemson
https://www.yelp.com/biz/sardis-den-clemson-3
https://www.yelp.com/biz/nicks-tavern-and-deli-clemson
https://www.yelp.com/biz/pixie-and-bills-clemson-2
https://www.yelp.com/biz/55-exchange-clemson
https://www.yelp.com/biz/bgr-the-burger-joint-clemson
https://www.yelp.com/biz/palmettos-smokehouse-and-oyster-bar-clemson
https://www.yelp.com/biz/tiger-town-tavern-clemson
https://www.yelp.com/biz/osaka-

## Challenge: 

Using this notebook, identify and print out the Yelp URL to all the restaurants in your hometown. 